## Import State-Wise GDP at Current Price

This data has been saved as Data1.csv. We first import the pandas library and then read the csv file.<br> We then use the DataFrame.head() method to check out the first few rows of the dataframe. 

In [25]:
import pandas as pd

df=pd.read_csv('Data1.csv')
df

,Items Description,Duration,Andhra Pradesh,Arunachal Pradesh,Assam,Bihar,Chhattisgarh,Goa,Gujarat,Haryana,...,Telangana,Tripura,Uttar Pradesh,Uttarakhand,West Bengal1,Andaman & Nicobar Islands,Chandigarh,Delhi,Puducherry,All_India GDP
0,GSDP - CURRENT PRICES (` in Crore),2011-12,379402.00,11063.00,143175.00,247144.00,158074.00,42367.00,615606.00,297539.00,...,359433.00,19208.00,724049.00,115523.00,NaN,3979.00,18768.00,343767.00,16818.00,8736039.00
1,GSDP - CURRENT PRICES (` in Crore),2012-13,411404.00,12547.00,156864.00,282368.00,177511.00,38120.00,724495.00,347032.00,...,401493.00,21663.00,822903.00,131835.00,NaN,4421.00,21609.00,391238.00,18875.00,9946636.00
2,GSDP - CURRENT PRICES (` in Crore),2013-14,464272.00,14602.00,177745.00,317101.00,206690.00,35921.00,807623.00,400662.00,...,452186.00,25593.00,944146.00,149817.00,NaN,5159.00,24787.00,443783.00,21870.00,11236635.00
3,GSDP - CURRENT PRICES (` in Crore),2014-15,526468.00,16761.00,198098.00,373920.00,234982.00,40633.00,895027.00,437462.00,...,511178.00,29667.00,1043371.00,161985.00,NaN,5721.00,27844.00,492424.00,24089.00,12433749.00
4,GSDP - CURRENT PRICES (` in Crore),2015-16,609934.00,18784.00,224234.00,413503.00,260776.00,45002.00,994316.00,485184.00,...,575631.00,NaN,1153795.00,184091.00,NaN,NaN,30304.00,551963.00,26533.00,13675331.00
5,GSDP - CURRENT PRICES (` in Crore),2016-17,699307.00,NaN,NaN,NaN,290140.00,NaN,NaN,547396.00,...,654294.00,NaN,NaN,NaN,NaN,NaN,NaN,622385.00,29557.00,15251028.00
6,(% Growth over previous year),2012-13,8.43,13.41,9.56,14.25,12.30,-10.02,17.69,16.63,...,11.70,12.78,13.65,14.12,NaN,11.13,15.14,13.81,12.23,13.86
7,(% Growth over previous year),2013-14,12.85,16.38,13.31,12.30,16.44,-5.77,11.47,15.45,...,12.63,18.14,14.73,13.64,NaN,16.68,14.71,13.43,15.87,12.97
8,(% Growth over previous year),2014-15,13.40,14.79,11.45,17.92,13.69,13.12,10.82,9.18,...,13.05,15.92,10.51,8.12,NaN,10.89,12.33,10.96,10.14,10.65
9,(% Growth over previous year),2015-16,15.85,12.07,13.19,10.59,10.98,10.75,11.09,10.91,...,12.61,NaN,10.58,13.65,NaN,NaN,8.84,12.09,10.15,9.99


This is not a very friendly format. Eventually we would want, one row for each state, with different metrics appearing as columns. Another problem is the first two rows carrying the items and the years.

We can merge them into one column. For example
>1.GSDP - CURRENT PRICES (` in Crore) and 2011-12 can be merged into GSDP_2012.<br>
>2.(% Growth over previous year) and 2012-13 can be merged into Growth_2012.<br>

To do this, we define a function merge_cols and apply them to the dataframe, so that we have a new column 'Metric'. Then we can drop the columns 'Items  Description','Duration'

In [26]:
def merge_cols(row):
    
    if row['Items  Description'].find('GSDP')==0:
        y='GSDP_'+row['Duration'][0:4]
        return y
    else:
        y='Growth_'+row['Duration'][0:4]
        return y

df['Metric']= df.apply(merge_cols, axis=1)
df=df.drop(['Items  Description','Duration'],axis=1)

We now transpose the dataframe, so that we can use the state-names are used as Index and the column 'Metric' becomes the header.

In [27]:
my_hdrs=list(df.Metric)
df.drop(['Metric'], axis=1,inplace=True)
my_df=df.T
my_df.columns=my_hdrs

We inspect the new data-frame my_df and shows a much more friendly format. We can use the state-names for merge operations coming up later.

In [30]:
my_df.head()

,GSDP_2011,GSDP_2012,GSDP_2013,GSDP_2014,GSDP_2015,GSDP_2016,Growth_2012,Growth_2013,Growth_2014,Growth_2015,Growth_2016
Andhra Pradesh,379402.0,411404.0,464272.0,526468.0,609934.0,699307.0,8.43,12.85,13.40,15.85,14.65
Arunachal Pradesh,11063.0,12547.0,14602.0,16761.0,18784.0,NaN,13.41,16.38,14.79,12.07,NaN
Assam,143175.0,156864.0,177745.0,198098.0,224234.0,NaN,9.56,13.31,11.45,13.19,NaN
Bihar,247144.0,282368.0,317101.0,373920.0,413503.0,NaN,14.25,12.30,17.92,10.59,NaN
Chhattisgarh,158074.0,177511.0,206690.0,234982.0,260776.0,290140.0,12.30,16.44,13.69,10.98,11.26
